In [ ]:
import sys
sys.path.append('../..')

In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from typing import Dict

In [ ]:
from survival_tests.survival_curves import create_scenario, create_survival_curves, plot_survival_funcs
from survival_tests.schedule import ground_truth_evaluation, survival_curve_from_schedule, termination_curve_from_train_data, compute_mean_runtime, schedule_termination_curve, normalized_par_k_values
from survival_tests.optimization import run_optimization, solution_to_schedule
from survival_tests.save_results import get_optimization_results, save_optimization_results, save_schedule_comparison, get_schedule_comparison, get_specific_optimization_results
from survival_tests.evaluation_of_schedules import normalized_par_k, not_included_indices
import survival_tests.optimization as survopt

Attributes

In [ ]:
scenario_name = 'QBF-2011'
save = False
PAR_K = 10
instance_id = 8

In [ ]:
scenario = create_scenario(scenario_name, filepath='../../survival_tests/results/workspaces/aslib/')
ALL_EVENT_TIMES, ALL_SURVIVAL_FUNCTIONS, CUTOFF, train_scenario, test_scenario = create_survival_curves(scenario, filename=scenario_name, filepath="survdata")

In [ ]:
indices = not_included_indices(test_scenario)
print(indices)
print(len(ALL_EVENT_TIMES) - len(indices[0]), len(ALL_SURVIVAL_FUNCTIONS) - len(indices[0]))
ALL_EVENT_TIMES = list(np.delete(ALL_EVENT_TIMES, indices, axis=0))
ALL_SURVIVAL_FUNCTIONS = list(np.delete(ALL_SURVIVAL_FUNCTIONS, indices, axis=0))
print(len(ALL_EVENT_TIMES), len(ALL_SURVIVAL_FUNCTIONS))

In [ ]:
plot_survival_funcs(np.array(ALL_EVENT_TIMES[instance_id]), np.array((ALL_SURVIVAL_FUNCTIONS[instance_id])), CUTOFF, save, filename= f"graphs/{scenario_name}_inputs")

In [ ]:
TET, TTF = termination_curve_from_train_data(test_scenario.performance_data, CUTOFF)
plot_survival_funcs(TET, TTF, CUTOFF)

In [ ]:
perf_values = np.min(test_scenario.performance_data.to_numpy(), axis=1)
current_penalty = 10 * test_scenario.algorithm_cutoff_time
np.where(perf_values==current_penalty)

In [ ]:
# solutions = {}
# num_alg = len(ALL_EVENT_TIMES[0])
# for instance_id in range(len(ALL_EVENT_TIMES)):
#     survopt.EVENT_TIMES = ALL_EVENT_TIMES[instance_id]
#     survopt.SURVIVAL_FUNCTIONS = ALL_SURVIVAL_FUNCTIONS[instance_id]
#     survopt.CUTOFF = CUTOFF
#     survopt.PAR_K = PAR_K

#     solution = run_optimization(num_alg, CUTOFF, max_num_iteration=300)
#     solutions[instance_id] = solution

# save_optimization_results(solutions, scenario_name, PAR_K)

solutions = get_optimization_results(scenario_name, PAR_K)
solutions

Visualize results

In [ ]:
perf_data = test_scenario.performance_data.to_numpy()
perf_data = np.delete(perf_data, indices, axis=0)

if test_scenario.feature_cost_data is not None:
    feature_cost_data = np.delete(test_scenario.feature_cost_data.to_numpy(), indices, axis=0)

solutions_schedules = get_specific_optimization_results(scenario_name, PAR_K, 'variable')
schedules = [solution_to_schedule(schedule, False) for schedule in solutions_schedules]

schedule_result_on_ground_truths = []
for idx in range(len(schedules)):
    gtvalues = pd.DataFrame(perf_data[idx]).to_dict()[0]
    gtvalues_schedule = ground_truth_evaluation(schedules[idx], gtvalues, CUTOFF, PAR_K * CUTOFF)

    # Add feature cost to the runtime
    if test_scenario.feature_cost_data is not None:
        feature_time = feature_cost_data[idx]
        accumulated_feature_time = np.sum(feature_time)
        schedule_result_on_ground_truths.append(gtvalues_schedule + accumulated_feature_time)
    else:
        schedule_result_on_ground_truths.append(gtvalues_schedule)

print("Unsolved Instances:", len(np.where(np.array(schedule_result_on_ground_truths) >= CUTOFF)[0]))
print(np.mean(schedule_result_on_ground_truths))
normalized_par_k(schedule_result_on_ground_truths, train_scenario, test_scenario, PAR_K)

In [ ]:
test_scenario.feature_cost_data

In [ ]:
print(schedules[0])

Testing Values

In [ ]:
from survival_tests.evaluation_of_schedules import mean_performance_of_singlebestssolver, mean_performance_of_virtualbestsolver
print(mean_performance_of_virtualbestsolver(test_scenario, PAR_K))
print(mean_performance_of_singlebestssolver(train_scenario, test_scenario, PAR_K))

Computing the Single Best Schedule on train data

In [ ]:
train_termination_event_times, train_termination_curve = termination_curve_from_train_data(train_scenario.performance_data, train_scenario.algorithm_cutoff_time)
plot_survival_funcs(train_termination_event_times, train_termination_curve, CUTOFF)

In [ ]:
num_algs = len(train_termination_event_times)
# survopt.SCENARIO = train_scenario
# survopt.CUTOFF = CUTOFF
# survopt.PAR_K = PAR_K
# solution = run_optimization(num_algs, CUTOFF, max_num_iteration=250, survival=False)
# Laufzeit steigt mit der Größe des Szenarios
solution = {'function': 9314.446271648194, 'variable':[3188.4784125 ,  222.01027302 ,1441.84967197, 1169.78489849 ,2509.10583516]}

In [ ]:
schedule = solution_to_schedule(solution)
static_schedule_event_times, static_schedule_termination_values = schedule_termination_curve(schedule, CUTOFF, test_scenario)

In [ ]:
plot_survival_funcs(static_schedule_event_times, static_schedule_termination_values, CUTOFF)

In [ ]:
test_termination_event_times, test_termination_curve = termination_curve_from_train_data(test_scenario.performance_data, test_scenario.algorithm_cutoff_time)
plot_survival_funcs(test_termination_event_times, test_termination_curve, CUTOFF)

In [ ]:
plot_survival_funcs(test_termination_event_times + static_schedule_event_times, test_termination_curve + static_schedule_termination_values, CUTOFF)

Overseeable statistic

In [ ]:
performance_algs = {f'Algorithm {i}':round(compute_mean_runtime(test_termination_event_times[i], test_termination_curve[i], CUTOFF, PAR_K)) for i in range(num_algs)}
performance_algs = pd.Series(performance_algs)
schedule_perf = {"SingleBestSchedule" : round(compute_mean_runtime(static_schedule_event_times[0], static_schedule_termination_values[0], CUTOFF, PAR_K))}
schedule_perf = pd.Series(schedule_perf)
result_table_on_inst = pd.concat([schedule_perf, performance_algs], sort=True)
result_table_on_inst

nPARK for Schedules

In [ ]:
vbsolver = mean_performance_of_virtualbestsolver(test_scenario, PAR_K)
normalized_par_k_values(np.mean(schedule_result_on_ground_truths), result_table_on_inst["SingleBestSchedule"], vbsolver)

SBSchedule in Comparison to SBSolver

In [ ]:
normalized_par_k(result_table_on_inst["SingleBestSchedule"], train_scenario, test_scenario, PAR_K)

Visualization of one testinstance

In [ ]:
event_times, survival_functions = survival_curve_from_schedule(schedules[instance_id], ALL_EVENT_TIMES[instance_id], ALL_SURVIVAL_FUNCTIONS[instance_id], CUTOFF)

In [ ]:
plot_survival_funcs([event_times], [survival_functions], CUTOFF, save, filename= f"graphs/{scenario_name}_outputsPAR{PAR_K}")

In [ ]:
plot_survival_funcs(list(ALL_EVENT_TIMES[instance_id]) + [event_times], list(ALL_SURVIVAL_FUNCTIONS[instance_id]) + [survival_functions], CUTOFF, save, filename= f"graphs/{scenario_name}_comparisonPAR{PAR_K}")

In [ ]:
perf_data[instance_id]

In [ ]:
schedules[instance_id]

In [ ]:
gtvalues = pd.DataFrame(perf_data[instance_id]).to_dict()[0]
ground_truth_evaluation(schedules[instance_id], gtvalues, CUTOFF, PAR_K * CUTOFF)

Visualization of one schedule's performance on test instances

In [ ]:
STET, STTF = schedule_termination_curve(schedules[instance_id], CUTOFF, test_scenario)

In [ ]:
plot_survival_funcs(STET, STTF, CUTOFF)

In [ ]:
plot_survival_funcs(TET + STET, TTF + STTF, CUTOFF)

Performance on every problem instance considering the survival functions

In [ ]:
num_instances = len(ALL_EVENT_TIMES)
num_algs = len(ALL_EVENT_TIMES[0])

In [ ]:
performance_on_inst = {f'Inst {inst}':{f'Algorithm {i}':round(compute_mean_runtime(ALL_EVENT_TIMES[instance_id][i], ALL_SURVIVAL_FUNCTIONS[instance_id][i], CUTOFF, inst)) for i in range(num_algs)} for inst in range(num_instances)}
inst_performance = pd.DataFrame(performance_on_inst)
inst_performance

In [ ]:
schedule_on_inst = {f'Inst {inst}': {"Schedule" : round(solutions[str(inst)]['function']) } for inst in range(num_instances)}
schedule_perf = pd.DataFrame(schedule_on_inst)
schedule_perf

In [ ]:
result_table_on_inst = pd.concat([schedule_perf, inst_performance], sort=True)
result_table_on_inst

Other overview

In [ ]:
PARS = [PAR_K]
algorithm_performace = {f'PAR{par}':{f'Algorithm {i}':round(compute_mean_runtime(ALL_EVENT_TIMES[instance_id][i], ALL_SURVIVAL_FUNCTIONS[instance_id][i], CUTOFF, par)) for i in range(len(ALL_EVENT_TIMES[instance_id]))} for par in PARS}
table_algorithms = pd.DataFrame(algorithm_performace)
table_algorithms

In [ ]:
objectives = {f'PAR{PAR_K}': {"Schedule" : round(solutions[str(instance_id)]['function'])}}
objective_table = pd.DataFrame(objectives)
objective_table

In [ ]:
result_table = pd.concat([objective_table, table_algorithms], sort=True)
result_table

In [ ]:
# save_schedule_comparison(scenario_name, PAR_K, scenario_properties=(ALL_EVENT_TIMES[instance_id], ALL_SURVIVAL_FUNCTIONS[instance_id], CUTOFF), curve_type='survival')
# get_schedule_comparison(scenario_name, curve_type='survival')

Performance considering the ground truth runtimes

In [ ]:
termination_algorithm_performace = {f'PAR{par}':{f'Algorithm {i}':round(compute_mean_runtime(TET[i], TTF[i], CUTOFF, par)) for i in range(len(TET))} for par in PARS}
termination_table_algorithms = pd.DataFrame(termination_algorithm_performace)
termination_table_algorithms

In [ ]:
termination_objectives = {f'PAR{PAR_K}': {"Schedule" : round(compute_mean_runtime(STET[0], STTF[0], CUTOFF, PAR_K))}}
termination_objective_table = pd.DataFrame(termination_objectives)
termination_objective_table

In [ ]:
termination_result_table = pd.concat([termination_objective_table, termination_table_algorithms], sort=True)
termination_result_table

In [ ]:
# save_schedule_comparison(scenario_name, PAR_K, scenario_properties=(TET, TTF, CUTOFF, STET, STTF), curve_type='termination')
# get_schedule_comparison(scenario_name, curve_type='termination')